In [59]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
import plotly.express as px
import folium

URL = [
    'https://www.chick-fil-a.com/locations/browse/al',
    'https://www.chick-fil-a.com/locations/browse/az',
    'https://www.chick-fil-a.com/locations/browse/ar',
    'https://www.chick-fil-a.com/locations/browse/ca',
    'https://www.chick-fil-a.com/locations/browse/co',
    'https://www.chick-fil-a.com/locations/browse/ct',
    'https://www.chick-fil-a.com/locations/browse/de',
    'https://www.chick-fil-a.com/locations/browse/fl',
    'https://www.chick-fil-a.com/locations/browse/ga',
    'https://www.chick-fil-a.com/locations/browse/id',
    'https://www.chick-fil-a.com/locations/browse/il',
    'https://www.chick-fil-a.com/locations/browse/in',
    'https://www.chick-fil-a.com/locations/browse/ia',
    'https://www.chick-fil-a.com/locations/browse/ks',
    'https://www.chick-fil-a.com/locations/browse/ky',
    'https://www.chick-fil-a.com/locations/browse/la',
    'https://www.chick-fil-a.com/locations/browse/me',
    'https://www.chick-fil-a.com/locations/browse/md',
    'https://www.chick-fil-a.com/locations/browse/ma',
    'https://www.chick-fil-a.com/locations/browse/mi',
    'https://www.chick-fil-a.com/locations/browse/mn',
    'https://www.chick-fil-a.com/locations/browse/ms',
    'https://www.chick-fil-a.com/locations/browse/mo',
    'https://www.chick-fil-a.com/locations/browse/mt',
    'https://www.chick-fil-a.com/locations/browse/ne',
    'https://www.chick-fil-a.com/locations/browse/nv',
    'https://www.chick-fil-a.com/locations/browse/nh',
    'https://www.chick-fil-a.com/locations/browse/nj',
    'https://www.chick-fil-a.com/locations/browse/nm',
    'https://www.chick-fil-a.com/locations/browse/ny',
    'https://www.chick-fil-a.com/locations/browse/nc',
    'https://www.chick-fil-a.com/locations/browse/nd',
    'https://www.chick-fil-a.com/locations/browse/oh',
    'https://www.chick-fil-a.com/locations/browse/ok',
    'https://www.chick-fil-a.com/locations/browse/or',
    'https://www.chick-fil-a.com/locations/browse/pa',
    'https://www.chick-fil-a.com/locations/browse/ri',
    'https://www.chick-fil-a.com/locations/browse/sc',
    'https://www.chick-fil-a.com/locations/browse/sd',
    'https://www.chick-fil-a.com/locations/browse/tn',
    'https://www.chick-fil-a.com/locations/browse/tx',
    'https://www.chick-fil-a.com/locations/browse/ut',
    'https://www.chick-fil-a.com/locations/browse/va',
    'https://www.chick-fil-a.com/locations/browse/dc',
    'https://www.chick-fil-a.com/locations/browse/wa',
    'https://www.chick-fil-a.com/locations/browse/wv',
    'https://www.chick-fil-a.com/locations/browse/wi',
    'https://www.chick-fil-a.com/locations/browse/wy',  
    
]

page = requests.get(URL[0])

soup = bs(page.content, 'html.parser')

results = soup.find(id="LocationFrameId")
#print(results.prettify())

#state = results.find_all("div", class_="state-locations-list")
stores = results.find_all("div", class_="location")
#print(stores)
loc=""
for element in stores:
    loc += element.find('p').text
    loc.strip()
    


def find_phone_num(text):
    regex = '.[0-9]{3}..[0-9]{3}.[0-9]{4}'
    data = re.findall(regex, str(text))
    return data

phones = find_phone_num(loc)


def find_st_address(text):
    # street address
    regex = '[0-9]+.[0-9a-zA-Z]+.[0-9a-zA-Z]+.[0-9a-zA-Z]+.[0-9a-zA-Z]+.[0-9a-zA-Z]+\s+[a-zA-Z]+.[a-zA-Z]+..[A-Z]{2}.[0-9]{5}'
    data = re.findall(regex, str(text))
    f = []
    for i in range(len(data)):
        f.append(data[i].replace('\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', ' '))
        
    return f

street = find_st_address(loc)

def lat_long(addy):
    funk = []
    for j in range(len(addy)):
        geolocator = Nominatim(user_agent= 'CFA')
        location = geolocator.geocode(addy[j])
        if location:
            lat=location.latitude
            lon=location.longitude
        else:
            lat=None
            lon=None
        funk.append((lat, lon))
        df=pd.DataFrame(funk)
    return df

df = lat_long(street)
print(df)
mean_lat= np.mean(df[0])
mean_lon= np.mean(df[1])

# create a base map centered around area
map_obj = folium.Map(location = [mean_lat, mean_lon], zoom_start = 100)
# create marker object for area, one by one for every location in data DataFrame
for i in range(0,df.shape[0]): # .shape[0] for Pandas DataFrame is the number of rows
    # create marker for location i 
    marker_obj = folium.Marker(location = [df[0][i],df[1][i]])
    # add marker to map
    marker_obj.add_to(map_obj)
# display map
# marker_obj = folium.Marker(location= [x[0],x[1]])
# marker_obj.add_to(map_obj)

map_obj


# geolocator = Nominatim(user_agent= 'CFA')
# location = geolocator.geocode(street[2])
# print(location.latitude, location.longitude)
# print(street, len(street))


            0          1
0   33.499662 -86.795602
1   30.676183 -88.174918
2         NaN        NaN
3   33.518251 -86.812574
4   33.238472 -86.802254
..        ...        ...
71  34.724858 -86.636027
72  33.493173 -86.813435
73  33.450346 -86.787746
74        NaN        NaN
75  33.447074 -86.824144

[76 rows x 2 columns]


ValueError: Location values cannot contain NaNs.